In [3]:
import sys, os
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

In [6]:
trainingDataFilename = 'trainingSet.csv'
testDataFilename = 'testSet.csv'
trainingSet = pd.read_csv(trainingDataFilename)
testSet = pd.read_csv(testDataFilename)
f = [0.025, 0.05, 0.075, 0.1, 0.15, 0.2]

In [13]:
trainingSet = trainingSet.sample(frac=1, random_state=18)
df_kfold = []
for i in range(10):
    df_kfold.append(trainingSet[i*520:(i+1)*520])
    
#print df_kfold[9]
nbc = {}
lr = {}
svm = {}

for t_frac in f:
    nbc[t_frac] = []
    lr[t_frac] = []
    svm[t_frac] = []
#print nbc

{0.2: [], 0.1: [], 0.025: [], 0.075: [], 0.15: [], 0.05: []}


In [ ]:
for t_frac in f:
    for i in range(10):
        # Partition the tarin and cv
        train_set_df = []
        for j in range(10):
            if j != i:
                train_set_df.append(df_kfold[j])
            else:
                test_set = df_kfold[j]
        
        train_set = pd.concat(train_set_df).sample(frac=t_frac, random_state=32)
        